# Load Required modules

In [1]:
import cv2
from cv2 import data
import numpy as np
import sys
import os
import time 
from sklearn import svm
import pickle

# First we initialize the directory paths to out dataset

In [2]:
base_dir = 'G:\\gestures\\SVM_dataset'
print(base_dir)

G:\gestures\SVM_dataset


In [3]:
train_dir = base_dir + '\\' + 'train'
test_dir = base_dir + '\\' + 'test'
print(train_dir)

G:\gestures\SVM_dataset\train


# Train the SVM model on our own dataset

In [ ]:
mem_x=0
mem_y=0
#importing various modules
dataset_path = train_dir

filenames=os.listdir(dataset_path)#path to the dataset of hands
print(filenames)
 
#reading dataset for training
X=[]
Y=[]
for i in filenames:
    temp_file_name = os.listdir(dataset_path + '\\' +i)
    for j in temp_file_name:
        # train the model on 200 * 200 pixel images
        img = cv2.imread(dataset_path + '\\' + i + '\\' + j, 0)
        #img=cv2.resize(img,(25,25))
        #res=cv2.resize(img,(25,25),interpolation=cv2.INTER_AREA)
        a = np.ravel(img) #covetring matrix into a vector
        Y.append(int(i))
        X.append(a)

x =np.array(X)
print(len(X),len(Y))
y=np.array(Y)
print(y)
c=1
h=0.02
clf = svm.SVC(kernel="linear",gamma='auto',C=c)#linear svm model
clf.fit(x,y)

In [ ]:
cap.release()

In [8]:
background = None
accumulated_weight = 0.7
mask_color = (0.0,0.0,0.0)

ROI_top = 100
ROI_bottom = 300
ROI_right = 300
ROI_left = 500

In [9]:
num_classes = {1:'1',2:'2',3:'3', 4:'4', 5:'5', 6:'6'}

## Below functions are helper functions

In [10]:
# This function is used to calculate accumulated_weights in the frame
def cal_accum_avg(frame, accumulated_weight):

    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None

    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [11]:
# This function segments the hand region found in the frame, if not found returns None.
def segment_hand(frame, threshold=50):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)

    
    _ , thresholded = cv2.threshold(diff, threshold, 255,cv2.THRESH_BINARY)
    
    edges = cv2.Canny(thresholded, threshold1= 50, threshold2=250)
    cv2.imshow('edges',thresholded)
    
     #Fetching contours in the frame (These contours can be of hand
    #or any other object in foreground) …

    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # If length of contours list = 0, means we didn't get any
    #contours...
    if len(contours) == 0:
        return None
    else:
        # The largest external contour should be the hand
        # contour_info = [(c, cv2.contourArea(c),) for c in contours[1]]

        #cntrs, heirs = cv2.findContours(thresholded.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

        contour_info = [(c, cv2.contourArea(c),) for c in contours]
        #for c in contours[1]:
        #    contour_info.append((c,cv2.contourArea(c),))
        
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        # Returning the hand segment(max contour) and the
  # thresholded image of hand and contour_info list
    return (thresholded, hand_segment_max_cont, contour_info)

# Prediction with the model

In [ ]:
cam = cv2.VideoCapture(0)
num_frames =0
cnt = 0
prev_num_frames = 0
while True:
    ret, frame = cam.read()

    # flipping the frame to prevent inverted image of captured
    #frame...
    
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    # ROI from the frame
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]

    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)


    if num_frames < 70:
        
        cal_accum_avg(gray_frame, accumulated_weight)
        
        cv2.putText(frame_copy, "FETCHING BACKGROUND...PLEASE WAIT",
  (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    else: 
        # segmenting the hand region
        hand = segment_hand(gray_frame)
        
        # Checking if we are able to detect the hand...
        if hand is not None:
            
            thresholded, hand_segment,contour_info = hand

            # Drawing contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right,
      ROI_top)], -1, (255, 0, 0),1)
            
            cv2.imshow("Thesholded Hand Image", thresholded)
            #res=cv2.resize(thresholded,(25,25),interpolation=cv2.INTER_AREA)
            #print(thresholded.shape)
            a_test = np.ravel(thresholded)
            result=int(clf.predict([a_test]))
            
            #print(res.shape)
            cv2.putText(frame_copy, str(result),
        (300, 45), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (0,0,255), 2)
       
            
                    
    # Draw ROI on frame_copy
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right,
    ROI_bottom), (255,128,0), 3)

    # incrementing the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.putText(frame_copy, "Indian sign language recognition_ _ _",
    (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Sign Detection", frame_copy)


    #prev = num_classes[np.argmax(pred) + 1]
    
    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()

In [ ]:
cam.release()

In [4]:
filename = 'SVM.sav'

In [ ]:

pickle.dump(clf, open(filename, 'wb'))
 
# some time later...

In [5]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [13]:
cam = cv2.VideoCapture(0)
num_frames =0
cnt = 0
prev_num_frames = 0
while True:
    ret, frame = cam.read()

    # flipping the frame to prevent inverted image of captured
    #frame...
    
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    # ROI from the frame
    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]

    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)


    if num_frames < 70:
        
        cal_accum_avg(gray_frame, accumulated_weight)
        
        cv2.putText(frame_copy, "FETCHING BACKGROUND...PLEASE WAIT",
  (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
    
    else: 
        # segmenting the hand region
        hand = segment_hand(gray_frame)
        
        # Checking if we are able to detect the hand...
        if hand is not None:
            
            thresholded, hand_segment,contour_info = hand

            # Drawing contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right,
      ROI_top)], -1, (255, 0, 0),1)
            
            cv2.imshow("Thesholded Hand Image", thresholded)
            #res=cv2.resize(thresholded,(25,25),interpolation=cv2.INTER_AREA)
            #print(thresholded.shape)
            a_test = np.ravel(thresholded)
            result=int(loaded_model.predict([a_test]))
            
            #print(res.shape)
            cv2.putText(frame_copy, str(result),
        (300, 45), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (0,0,255), 2)
       
            
                    
    # Draw ROI on frame_copy
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right,
    ROI_bottom), (255,128,0), 3)

    # incrementing the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.putText(frame_copy, "Indian sign language recognition_ _ _",
    (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    cv2.imshow("Sign Detection", frame_copy)


    #prev = num_classes[np.argmax(pred) + 1]
    
    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

# Release the camera and destroy all the windows
cam.release()
cv2.destroyAllWindows()

In [14]:
cam.release()